ref https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/


In [3]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:

import re
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import EarlyStopping

import numpy as np
import random
import os
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

from numpy import argmax
from sklearn.preprocessing import LabelEncoder

nltk.download('stopwords')
STOPWORDS = set(stopwords.words('english'))

print(tf.__version__)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2.3.0


In [5]:
df = pd.read_csv('/content/drive/My Drive/reduced30k.tsv', sep='\t')


In [6]:

from numpy import argmax


#newdf  = df.sample(frac=.1)
newdf = df.copy()
newdf['Text'] =newdf['title'].str.cat(newdf['selftext'], sep=' ')
newdf = newdf[['subreddit', 'Text']]
newdf.columns = ['Class Name', 'Text']



In [7]:
#need convert the labels to numeric
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(newdf['Class Name'])
print(integer_encoded, len(label_encoder.classes_))   
       


[164 107 657 ... 802 339 421] 1013


In [8]:

training_portion = .8
train_size = int(newdf.shape[0] * training_portion)

train,test = newdf[0: train_size],newdf[train_size:]
print(train.shape,test.shape)

test.head()

(243120, 2) (60780, 2)


,Class Name,Text
243120,learnmachinelearning,Starting an image recognition network? Hey all...
243121,GenderCritical,Women = infants I’m sure someone has covered t...
243122,PlasticSurgery,I am getting a rhinoplasty to fix breathing af...
243123,DebateAltRight,"""Open borders, but only for white countries"" S..."
243124,pihole,Web Interface (Lighttpd) stops working Some mo...


In [9]:
vocab_size = 10000
#vocab_size = 5000
embedding_dim = 13
max_length = 200
trunc_type='post'
oov_tok = "<OOV>"

train_texts = train['Text'].tolist()
test_texts = test['Text'].tolist()

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_texts)   #for train, 

word_index = tokenizer.word_index  

X_train_sequences = tokenizer.texts_to_sequences(train_texts)     # sequences is a list of seq
X_train_padded = pad_sequences(X_train_sequences,maxlen=max_length, truncating=trunc_type)  #training X

##
X_test_sequences = tokenizer.texts_to_sequences(test_texts)     #testing X
X_test_padded = pad_sequences(X_test_sequences,maxlen=max_length)            #X_test_padded


train_y = train['Class Name'].tolist()
test_y = test['Class Name'].tolist()

# transform labels into int
le  = LabelEncoder()
le.fit(newdf['Class Name'])


train_y = le.transform(train_y)
test_y = le.transform(test_y)
train_y = tf.keras.utils.to_categorical(train_y)    # https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical
test_y = tf.keras.utils.to_categorical(test_y)

In [10]:
len(train_y) , len(X_train_sequences)

len(word_index)

424272

In [ ]:

woahvicky = EarlyStopping(patience=4, restore_best_weights=True)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
#    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.LSTM(100, dropout=0.2),
    tf.keras.layers.Dense(1013, activation='softmax')   ## this matches the number of categories
    ])

model.summary()
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

NUM_EPOCHS = 1000

history = model.fit(X_train_padded,train_y,
                    batch_size=32, 
                    epochs=NUM_EPOCHS,
                    validation_data=(X_test_padded,test_y),
                    callbacks= woahvicky)




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 13)          130000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         8448      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               91600     
_________________________________________________________________
dense_1 (Dense)              (None, 1013)              102313    
Total params: 332,361
Trainable params: 332,361
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
7598/7598 [==============================] - 78s 10ms/step - loss: 6.7371 - accuracy: 0.0029 - val_lo

In [ ]:
model.save(
    'cnn_model1.h5', overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None
)